# Pair Programming ETL II

Tendréis que usar el csv attacks_limpieza_completa que tenéis adjunto abajo.
En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:
1. El csv con los ataques de tiburones que hemos estado limpiando hasta ahora, el que os hemos adjuntado (attacks_limpieza_completa). Sentiros libres de usar vuestros propios csv en caso de que queráis.
2. El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer.

El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:
- Cargaremos los dos ficheros de datos
- Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:
    -  USA
    -  Australia
    -  New Zealand
    -  South Africa
    -  Papua New Guinea
- Del dataframe de los datos climáticos seleccionaremos todas las columnas.
- Cuando ya tengamos todos los datos deseados juntaremos los dos csv.
    - Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.
    - Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

In [53]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

import requests
import pandas as pd
import numpy as np



In [54]:
pd.options.display.max_columns = None

In [55]:
df_attack = pd.read_csv("../datos/datos_limpiezaV.csv", index_col=0)
df_clima = pd.read_csv("../datos/datos_clima.csv",index_col=0)

In [56]:
df_clima.head()

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 10}, {'layer': '900m...","[{'layer': '950mb', 'direction': 175, 'speed':...",27,-4,10,-9999,rain,2,0,150,2,usa
1,6,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 195, 'speed':...",26,-1,10,-9999,rain,2,0,160,1,usa
2,9,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 180, 'speed':...",27,-1,10,-9999,rain,3,0,180,2,usa
3,12,7,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 180, 'speed':...",27,-1,11,-9999,rain,3,0,180,3,usa
4,15,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 185, 'speed':...",27,-1,11,-9999,rain,3,0,185,3,usa


In [57]:
df_attack.head()

,year,country,age,species,date,month,fatal,sex,main_species,age_numbers,age_numbers_mean,sex_moda
0,2018.0,usa,57,White shark,25-Jun-2018,Jun,n,F,white_shark,57.0,57.0,F
1,2018.0,brazil,18,Tiger shark,03-Jun-2018,Jun,y,M,tiger_shark,18.0,18.0,M
2,2018.0,usa,15,"Bull shark, 6'",26-May-2018,May,n,M,bull_shark,15.0,15.0,M
3,2018.0,australia,32,Grey reef shark,24-May-2018,May,n,M,grey_shark,32.0,32.0,M
4,2018.0,england,21,Invalid incident,13-May-2018,May,n,M,unspecified,21.0,21.0,M


In [58]:
attaques = df_attack[(df_attack["country"] == "usa") | (df_attack["country"] == "australia") | (df_attack["country"] == "papua new guinea") | (df_attack["country"] == "new zealand")| (df_attack["country"] == "south africa")]
attaques.head()
                     

,year,country,age,species,date,month,fatal,sex,main_species,age_numbers,age_numbers_mean,sex_moda
0,2018.0,usa,57,White shark,25-Jun-2018,Jun,n,F,white_shark,57.0,57.0,F
2,2018.0,usa,15,"Bull shark, 6'",26-May-2018,May,n,M,bull_shark,15.0,15.0,M
3,2018.0,australia,32,Grey reef shark,24-May-2018,May,n,M,grey_shark,32.0,32.0,M
6,2018.0,australia,60,3 m shark,25-Apr-2018,Apr,n,M,unspecified,60.0,60.0,M
8,2018.0,south africa,33,"White shark, 2.5 m",22-Apr-2018,Apr,n,M,white_shark,33.0,33.0,M


In [59]:
df_clima.reset_index(inplace=True)

In [60]:

import ast

df_clima['rh_profile']= df_clima['rh_profile'].apply(ast.literal_eval)

# una vez que tengamos la columna cambiada, una fantasía de Pandas es que si hago un apply sobre una columna cuyos valores son diccionarios o listas nos va a genererar una columna con los valores de los diccionarios o listas. Donde cada columna será key del diccionario o cada elemento de la lista. 


#x = df_clima['wind_profile'].apply(pd.Series)


# nos creamos un dataframe nuevo con el resultado de la información de una de las columnas separadas por columnas. Esto nos va a devolver un dataframe donde cada fila será una celda del dataframe anterior. 
x = df_clima['rh_profile'].apply(pd.Series) 

# ¿Qué es lo que ocurre cuando hacemos esto?
# Nos ha creado tantas columnas como valores tuvieramos en la lista. Donde columna es, en este caso, un diccionario (porque nuestra lista esta compuesta por distintos diccionarios)

# Ok, hemos conseguido desempaquetar la información de la lista en distintas columnas. Ahora tenemos que despempaquetar la información de los diccionarios en distintas columnas. En este caso, lo que querremos es que las key de los diccionarios sean los nombres de las columnas y los values los valores de las celdas del dataframe. Volveremos a seguir entonces la misma lógica que antes con el apply, pero en este caso necesitamos hacerlo para todo el dataframe (que es x): 

# Por eso empezamos con un for para iterar por cada una de las columnas. 
for i in range(len(x.columns)): 

    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(x[i].apply(pd.Series)["layer"][0]) 

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(x[i].apply(pd.Series)["rh"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df_clima.insert(i, nombre, valores)

# una vez que hayamos hecho esto para las dos columnas ya podremos hacer el gropuby para después unir toda la información. 

In [61]:
df_clima.head()

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,10,10,11,11,12,13,12,9,8,10,15,15,16,16,12,9,0,3,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 10}, {'layer': '900m...","[{'layer': '950mb', 'direction': 175, 'speed':...",27,-4,10,-9999,rain,2,0,150,2,usa
1,11,12,12,12,12,12,11,10,8,9,12,15,16,16,15,13,1,6,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 195, 'speed':...",26,-1,10,-9999,rain,2,0,160,1,usa
2,11,12,12,12,13,14,14,13,10,7,12,12,10,15,16,15,2,9,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 180, 'speed':...",27,-1,10,-9999,rain,3,0,180,2,usa
3,11,12,11,11,13,15,16,15,9,6,4,5,14,16,16,10,3,12,7,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 180, 'speed':...",27,-1,11,-9999,rain,3,0,180,3,usa
4,13,12,11,11,13,15,15,15,13,6,4,6,10,6,4,6,4,15,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 185, 'speed':...",27,-1,11,-9999,rain,3,0,185,3,usa


In [62]:
#Para WIND

In [63]:

df_clima['wind_profile']= df_clima['wind_profile'].apply(ast.literal_eval)

# una vez que tengamos la columna cambiada, una fantasía de Pandas es que si hago un apply sobre una columna cuyos valores son diccionarios o listas nos va a genererar una columna con los valores de los diccionarios o listas. Donde cada columna será key del diccionario o cada elemento de la lista. 


y = df_clima['wind_profile'].apply(pd.Series)

for i in range(len(y.columns)): 
    nombre = "winddirection_" + str(y[i].apply(pd.Series)["layer"][0])

    valores = list(y[i].apply(pd.Series)["direction"])

    df_clima.insert(i, nombre, valores)


In [64]:


for i in range(len(y.columns)): 

    #aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "wind_speed_" + str(y[i].apply(pd.Series)["layer"][0]) 

    #hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(y[i].apply(pd.Series)["speed"])

    #usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df_clima.insert(i, nombre, valores)



In [65]:
df_clima.head()

,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,wind_speed_450mb,wind_speed_400mb,wind_speed_350mb,wind_speed_300mb,wind_speed_250mb,wind_speed_200mb,winddirection_950mb,winddirection_900mb,winddirection_850mb,winddirection_800mb,winddirection_750mb,winddirection_700mb,winddirection_650mb,winddirection_600mb,winddirection_550mb,winddirection_500mb,winddirection_450mb,winddirection_400mb,winddirection_350mb,winddirection_300mb,winddirection_250mb,winddirection_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,2,2,2,2,2,3,3,4,4,5,3,4,4,4,5,5,175,180,165,125,95,85,85,80,65,45,80,100,105,110,110,115,10,10,11,11,12,13,12,9,8,10,15,15,16,16,12,9,0,3,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 10}, {'layer': '900m...","[{'layer': '950mb', 'direction': 175, 'speed':...",27,-4,10,-9999,rain,2,0,150,2,usa
1,2,2,1,2,2,3,3,4,4,5,3,4,5,5,5,5,195,180,105,135,120,95,95,90,70,50,75,80,90,105,115,125,11,12,12,12,12,12,11,10,8,9,12,15,16,16,15,13,1,6,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 195, 'speed':...",26,-1,10,-9999,rain,2,0,160,1,usa
2,2,2,2,2,2,3,3,3,3,4,5,4,5,5,5,5,180,145,65,85,100,95,95,85,70,55,80,95,80,110,120,130,11,12,12,12,13,14,14,13,10,7,12,12,10,15,16,15,2,9,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 180, 'speed':...",27,-1,10,-9999,rain,3,0,180,2,usa
3,3,2,2,2,2,3,3,3,3,4,5,5,5,5,5,5,180,195,355,20,35,60,90,105,105,70,60,60,75,100,150,160,11,12,11,11,13,15,16,15,9,6,4,5,14,16,16,10,3,12,7,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 180, 'speed':...",27,-1,11,-9999,rain,3,0,180,3,usa
4,3,2,2,2,2,2,2,3,4,4,5,5,5,5,4,4,185,200,315,345,355,30,105,105,120,75,55,55,75,110,140,150,13,12,11,11,13,15,15,15,13,6,4,6,10,6,4,6,4,15,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 185, 'speed':...",27,-1,11,-9999,rain,3,0,185,3,usa


In [66]:
df_clima = df_clima.drop(["rh_profile", "wind_profile"], axis=1) #Borramos las columnas 

In [69]:
df_group =df_clima.groupby('country').mean()
df_group.reset_index()

,country,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,wind_speed_450mb,wind_speed_400mb,wind_speed_350mb,wind_speed_300mb,wind_speed_250mb,wind_speed_200mb,winddirection_950mb,winddirection_900mb,winddirection_850mb,winddirection_800mb,winddirection_750mb,winddirection_700mb,winddirection_650mb,winddirection_600mb,winddirection_550mb,winddirection_500mb,winddirection_450mb,winddirection_400mb,winddirection_350mb,winddirection_300mb,winddirection_250mb,winddirection_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,australia,2.390625,2.125,2.390625,2.9375,3.15625,3.140625,3.125,3.21875,3.5,4.0625,3.953125,3.484375,3.296875,3.3125,4.0625,4.34375,182.65625,159.53125,95.390625,64.21875,64.6875,72.8125,86.015625,94.140625,102.34375,95.0,78.046875,76.015625,63.75,107.8125,176.796875,201.640625,12.59375,10.40625,8.09375,6.953125,7.921875,9.328125,10.171875,11.046875,10.796875,4.1875,-0.015625,-0.015625,2.0625,5.03125,6.171875,5.8125,31.5,97.5,5.625,-9999.0,-9999.0,-9999.0,26.765625,-1.609375,10.5625,-9999.0,2.96875,0.0,185.859375,2.4375
1,new zealand,2.390625,2.125,2.390625,2.9375,3.15625,3.140625,3.125,3.21875,3.5,4.0625,3.953125,3.484375,3.296875,3.3125,4.0625,4.34375,182.65625,159.53125,95.390625,64.21875,64.6875,72.8125,86.015625,94.140625,102.34375,95.0,78.046875,76.015625,63.75,107.8125,176.796875,201.640625,12.59375,10.40625,8.09375,6.953125,7.921875,9.328125,10.171875,11.046875,10.796875,4.1875,-0.015625,-0.015625,2.0625,5.03125,6.171875,5.8125,31.5,97.5,5.625,-9999.0,-9999.0,-9999.0,26.765625,-1.609375,10.5625,-9999.0,2.96875,0.0,185.859375,2.4375
2,papua new guinea,2.390625,2.125,2.390625,2.9375,3.15625,3.140625,3.125,3.21875,3.5,4.0625,3.953125,3.484375,3.296875,3.3125,4.0625,4.34375,182.65625,159.53125,95.390625,64.21875,64.6875,72.8125,86.015625,94.140625,102.34375,95.0,78.046875,76.015625,63.75,107.8125,176.796875,201.640625,12.59375,10.40625,8.09375,6.953125,7.921875,9.328125,10.171875,11.046875,10.796875,4.1875,-0.015625,-0.015625,2.0625,5.03125,6.171875,5.8125,31.5,97.5,5.625,-9999.0,-9999.0,-9999.0,26.765625,-1.609375,10.5625,-9999.0,2.96875,0.0,185.859375,2.4375
3,south africa,2.390625,2.125,2.390625,2.9375,3.15625,3.140625,3.125,3.21875,3.5,4.0625,3.953125,3.484375,3.296875,3.3125,4.0625,4.34375,182.65625,159.53125,95.390625,64.21875,64.6875,72.8125,86.015625,94.140625,102.34375,95.0,78.046875,76.015625,63.75,107.8125,176.796875,201.640625,12.59375,10.40625,8.09375,6.953125,7.921875,9.328125,10.171875,11.046875,10.796875,4.1875,-0.015625,-0.015625,2.0625,5.03125,6.171875,5.8125,31.5,97.5,5.625,-9999.0,-9999.0,-9999.0,26.765625,-1.609375,10.5625,-9999.0,2.96875,0.0,185.859375,2.4375
4,usa,2.390625,2.125,2.390625,2.9375,3.15625,3.140625,3.125,3.21875,3.5,4.0625,3.953125,3.484375,3.296875,3.3125,4.0625,4.34375,182.65625,159.53125,95.390625,64.21875,64.6875,72.8125,86.015625,94.140625,102.34375,95.0,78.046875,76.015625,63.75,107.8125,176.796875,201.640625,12.59375,10.40625,8.09375,6.953125,7.921875,9.328125,10.171875,11.046875,10.796875,4.1875,-0.015625,-0.015625,2.0625,5.03125,6.171875,5.8125,31.5,97.5,5.625,-9999.0,-9999.0,-9999.0,26.765625,-1.609375,10.5625,-9999.0,2.96875,0.0,185.859375,2.4375


In [73]:
df_final = pd.merge(df_group, df_attack, how="inner", right_on="country", left_on="country")


In [74]:
df_final.head(5)

,country,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,wind_speed_450mb,wind_speed_400mb,wind_speed_350mb,wind_speed_300mb,wind_speed_250mb,wind_speed_200mb,winddirection_950mb,winddirection_900mb,winddirection_850mb,winddirection_800mb,winddirection_750mb,winddirection_700mb,winddirection_650mb,winddirection_600mb,winddirection_550mb,winddirection_500mb,winddirection_450mb,winddirection_400mb,winddirection_350mb,winddirection_300mb,winddirection_250mb,winddirection_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,year,age,species,date,month,fatal,sex,main_species,age_numbers,age_numbers_mean,sex_moda
0,australia,2.390625,2.125,2.390625,2.9375,3.15625,3.140625,3.125,3.21875,3.5,4.0625,3.953125,3.484375,3.296875,3.3125,4.0625,4.34375,182.65625,159.53125,95.390625,64.21875,64.6875,72.8125,86.015625,94.140625,102.34375,95.0,78.046875,76.015625,63.75,107.8125,176.796875,201.640625,12.59375,10.40625,8.09375,6.953125,7.921875,9.328125,10.171875,11.046875,10.796875,4.1875,-0.015625,-0.015625,2.0625,5.03125,6.171875,5.8125,31.5,97.5,5.625,-9999.0,-9999.0,-9999.0,26.765625,-1.609375,10.5625,-9999.0,2.96875,0.0,185.859375,2.4375,2018.0,32,Grey reef shark,24-May-2018,May,n,M,grey_shark,32.0,32.0,M
1,australia,2.390625,2.125,2.390625,2.9375,3.15625,3.140625,3.125,3.21875,3.5,4.0625,3.953125,3.484375,3.296875,3.3125,4.0625,4.34375,182.65625,159.53125,95.390625,64.21875,64.6875,72.8125,86.015625,94.140625,102.34375,95.0,78.046875,76.015625,63.75,107.8125,176.796875,201.640625,12.59375,10.40625,8.09375,6.953125,7.921875,9.328125,10.171875,11.046875,10.796875,4.1875,-0.015625,-0.015625,2.0625,5.03125,6.171875,5.8125,31.5,97.5,5.625,-9999.0,-9999.0,-9999.0,26.765625,-1.609375,10.5625,-9999.0,2.96875,0.0,185.859375,2.4375,2018.0,60,3 m shark,25-Apr-2018,Apr,n,M,unspecified,60.0,60.0,M
2,australia,2.390625,2.125,2.390625,2.9375,3.15625,3.140625,3.125,3.21875,3.5,4.0625,3.953125,3.484375,3.296875,3.3125,4.0625,4.34375,182.65625,159.53125,95.390625,64.21875,64.6875,72.8125,86.015625,94.140625,102.34375,95.0,78.046875,76.015625,63.75,107.8125,176.796875,201.640625,12.59375,10.40625,8.09375,6.953125,7.921875,9.328125,10.171875,11.046875,10.796875,4.1875,-0.015625,-0.015625,2.0625,5.03125,6.171875,5.8125,31.5,97.5,5.625,-9999.0,-9999.0,-9999.0,26.765625,-1.609375,10.5625,-9999.0,2.96875,0.0,185.859375,2.4375,2018.0,10,Wobbegong shark,14-Mar-2018,Mar,n,M,unspecified,10.0,10.0,M
3,australia,2.390625,2.125,2.390625,2.9375,3.15625,3.140625,3.125,3.21875,3.5,4.0625,3.953125,3.484375,3.296875,3.3125,4.0625,4.34375,182.65625,159.53125,95.390625,64.21875,64.6875,72.8125,86.015625,94.140625,102.34375,95.0,78.046875,76.015625,63.75,107.8125,176.796875,201.640625,12.59375,10.40625,8.09375,6.953125,7.921875,9.328125,10.171875,11.046875,10.796875,4.1875,-0.015625,-0.015625,2.0625,5.03125,6.171875,5.8125,31.5,97.5,5.625,-9999.0,-9999.0,-9999.0,26.765625,-1.609375,10.5625,-9999.0,2.96875,0.0,185.859375,2.4375,2018.0,69,3.5 m shark,9-Mar-2018,Mar,n,M,unspecified,27.9,27.9,M
4,australia,2.390625,2.125,2.390625,2.9375,3.15625,3.140625,3.125,3.21875,3.5,4.0625,3.953125,3.484375,3.296875,3.3125,4.0625,4.34375,182.65625,159.53125,95.390625,64.21875,64.6875,72.8125,86.015625,94.140625,102.34375,95.0,78.046875,76.015625,63.75,107.8125,176.796875,201.640625,12.59375,10.40625,8.09375,6.953125,7.921875,9.328125,10.171875,11.046875,10.796875,4.1875,-0.015625,-0.015625,2.0625,5.03125,6.171875,5.8125,31.5,97.5,5.625,-9999.0,-9999.0,-9999.0,26.765625,-1.609375,10.5625,-9999.0,2.96875,0.0,185.859375,2.4375,2018.0,18,1.8 m shark,9-Mar-2018,Mar,n,M,unspecified,18.0,18.0,M


In [76]:
df_final.to_csv("../datos/datos_ETL2.csv")

